In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima_model import ARIMA

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend
from tensorflow.keras import Model, Sequential
from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Input, SimpleRNN, Embedding, Dense, TimeDistributed, GRU, \
                                    Dropout, Bidirectional, Conv1D, BatchNormalization

print(tf.keras.__version__)
print(tf.__version__)

/Users/zihaoxu/ENTER/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


2.2.4-tf
2.0.0


In [11]:
flu_df = pd.read_csv('../data/flu_data.csv').sort_values('date')
print(flu_df.shape)
flu_df.head()

(461, 54)


,date,Texas,Louisiana,Washington,New Jersey,Arkansas,Montana,North Dakota,Virgin Islands,Nebraska,...,Nevada,Idaho,Utah,Virginia,New York,District of Columbia,Oklahoma,South Dakota,New York City,Colorado
0,2010-40,2.06514,2.28108,0.510041,2.31534,0.696056,0.037119,0.671355,NaN,2.199780,...,1.66683,0.414938,0.747696,1.14343,0.262990,2.80877,0.624179,1.102750,1.18222,0.660684
1,2010-41,2.09394,2.04776,1.040070,2.47836,0.674157,0.180963,0.721961,NaN,2.735320,...,1.63633,1.457980,0.410939,1.23653,0.369729,2.89079,0.958678,0.793794,1.40230,0.628621
2,2010-42,2.10072,2.42974,0.904393,2.24812,0.514217,0.185874,0.567477,NaN,3.311030,...,1.50860,1.408140,0.440583,1.22545,0.362121,2.41042,2.070390,0.791023,1.29124,0.804020
3,2010-43,2.20655,1.92854,0.955110,2.79286,0.413650,0.036390,0.339239,NaN,2.057610,...,2.14451,1.280070,0.755957,1.26902,0.358981,3.11632,1.670730,0.801311,1.30236,0.909658
4,2010-44,2.36381,2.80934,0.786370,1.82877,1.090280,0.040519,0.713749,NaN,0.376712,...,1.76761,1.827240,0.651859,1.26547,0.527357,2.99118,1.628450,0.779833,1.51508,0.971705


In [6]:
texas_google = pd.read_csv('../data/texas_google.csv')
newyork_google = pd.read_csv('../data/newyork_google.csv')
print(texas_google.shape)
print(newyork_google.shape)

(120, 97)
(120, 97)


In [7]:
texas_google.head()

,date,flu,treating flu,medicine for the flu,high fever,is the flu contagious,get over the flu,medicine for flu,how to get rid of the flu,fever cough,...,flu a symptoms,flu in children,do i have the flu,i have the flu,influenza incubation period,flu season,oscillococcinum,what is the flu,low body,remedies for the flu
0,2010-01-01,11,0,6,53,6,0,6,6,25,...,17,0,4,4,0,4,2,6,29,0
1,2010-02-01,9,0,6,32,6,7,7,7,23,...,0,9,0,2,0,2,3,6,39,0
2,2010-03-01,6,6,0,39,6,0,0,0,15,...,12,0,8,2,0,0,2,8,32,0
3,2010-04-01,6,0,0,74,0,0,13,0,13,...,4,0,0,2,2,3,2,17,19,0
4,2010-05-01,3,0,0,71,0,0,0,0,13,...,0,4,0,2,0,0,2,4,30,4


In [8]:
pd.merge(flu_df, texas_google, on = 'date', how = 'left')

,date,Texas,Louisiana,Washington,New Jersey,Arkansas,Montana,North Dakota,Virgin Islands,Nebraska,...,flu a symptoms,flu in children,do i have the flu,i have the flu,influenza incubation period,flu season,oscillococcinum,what is the flu,low body,remedies for the flu
0,2010-40,2.06514,2.281080,0.510041,2.31534,0.696056,0.037119,0.671355,NaN,2.199780,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-41,2.09394,2.047760,1.040070,2.47836,0.674157,0.180963,0.721961,NaN,2.735320,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-42,2.10072,2.429740,0.904393,2.24812,0.514217,0.185874,0.567477,NaN,3.311030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-43,2.20655,1.928540,0.955110,2.79286,0.413650,0.036390,0.339239,NaN,2.057610,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-44,2.36381,2.809340,0.786370,1.82877,1.090280,0.040519,0.713749,NaN,0.376712,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2010-45,2.51778,2.085770,1.067240,2.20310,1.197360,0.243902,0.758110,NaN,2.961080,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2010-46,2.24622,2.815060,1.331320,2.13716,1.554620,0.109930,0.644192,NaN,2.423030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2010-47,2.59866,2.927060,1.873330,2.41758,2.025990,0.537057,0.590551,NaN,0.614754,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2010-48,2.56617,2.664820,0.981767,2.60116,1.392950,0.038052,0.347088,NaN,2.815700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2010-49,2.40335,2.804860,0.964630,2.65371,1.020640,0.035945,0.237530,NaN,2.462120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
